In [ ]:
# Interactive Figure
%matplotlib ipympl 
# Non interactive
# %matplotlib inline 

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig
from qualang_tools.results import fetching_tool
from qualang_tools.loops import from_array
import config_00 as config
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def addjob(qmprog, qm):
    # Add a QUA program to the OPX queue, which compiles it and executes it
    job = qm.queue.add(qmprog)
    # Wait for job to be loaded
    while job.status=="loading":
        print("Job is loading...")
        time.sleep(0.1)
    # Wait until job is running
    time.sleep(0.1)
    while job.status=="pending":
        q = job.position_in_queue()
        if q>0:
            print("Position in queue",q,end='\r')
        time.sleep(0.1)
    job=job.wait_for_execution()
    print("\nJob is running")
    return job

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Get running QM instance

In [ ]:
# Get the QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Send Pulses and Visualize them

In [ ]:
# This block is using QUA directives which are compiled to the FPGA
with program() as prog:
    # Variable declaration
    adc_stream = declare_stream(adc_trace=True)
    # Start measurement
    measure('readout', 'scope', adc_stream=adc_stream)
    # Pulse sequence
    play('pulse','qubit')
    wait(24*u.ns)
    play('pulse','qubit')

    # Select data to be sent to the client 
    with stream_processing():
        adc_stream.input1().save('adc_1')
        adc_stream.input2().save('adc_2')

In [ ]:
# Run the code and fetch results
job = addjob(prog, qm)
res = fetching_tool(job, ['adc_1','adc_2'])
adc_1, adc_2 = res.fetch_all()
print('Job done')

In [ ]:
# Plot
t = np.arange(config.readout_len)
fig,ax=plt.subplots()
ax.plot(t,adc_1,t,adc_2)

# Simulate the QUA code

In [ ]:
plt.figure()
# Simulates the QUA program for the specified duration
simulation_config = SimulationConfig(duration=config.readout_len//4)  # In clock cycles = 4ns
# Simulate blocks python until the simulation is done
job = qmm.simulate(config.config, prog, simulation_config)
# Get the simulated samples
samples = job.get_simulated_samples()
# Plot the simulated samples
samples.con1.plot()
# Get the waveform report object
waveform_report = job.get_simulated_waveform_report()
# Cast the waveform report to a python dictionary
waveform_dict = waveform_report.to_dict()
# Visualize and save the waveform report
waveform_report.create_plot(samples, plot=True)

# Practice
- Link to the Quantum Machine documentation [https://docs.quantum-machines.co/latest/](https://docs.quantum-machines.co/latest/)
- Read the doc about the [`play`](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.play) command

## Exercise 1: Phase evolution
- Play two pulses one after each other without waiting time. Are the traces continuous? Why?
- Dephase the second pulse by $\pi/2$ using a [frame_rotation](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.frame_rotation) command. Add a delay again between the pulses, observe the phase evolution. 
- Put back the delay to 0 and instead of using the frame_rotation, depahse the second pulse by $\pi/2$ using the [amp directive](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.amp) with four parameters 
 
## Exercise 2: Pulse timing 
- Change the pulse duration to 100ns using the `duration` keyword in the [`play`](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.play) command
- Read the doc on [timing](https://docs.quantum-machines.co/latest/docs/Guides/timing_in_qua/)
- Modify the sequence to create two pulses separated by 500ns and set the measurement window such that only the second pulse is visible and arrives at 250ns

## Exercise 3: For loops
- Write a loop using [`with for_`](https://docs.quantum-machines.co/latest/docs/Guides/features/#loops) in order to create a train of 5 pulses.

## Exercise 4: Arbitrary Pulses
- In the play command, replace the `pulse` by `gaussian` and observe the results (remove any duration keyword if present), check the `config_00.py` file to see how arbitrary pulses are built